# Flight Agent: Your Travel Planner

Time for an exciting use case. Now is the time to use all the knowledge we have gathered so far to build a complete AI Agent ourselves.

## Plan of Attack:

1. Import libraries
2. Define our tools
3. Define our tools schema
4. Call the OpenAI Responses API
5. Handle tool calls
6. Chat logic
7. Gradio Interface

## Step 1: Import libraries

In [ ]:
import os
from dotenv import load_dotenv
import json
from openai import OpenAI
from utils import function_to_tool
from IPython.display import display, Markdown
import gradio as gr

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY is not set in the environment variables.")

openai_client = OpenAI()

## Step 2: Define our tools

In [ ]:
query = "I want to travel alone from abu dhabi to tokyo tomorrow and will be back in 2 days. what are the best flights available for economy class?"

# Flight search tool
response = openai_client.responses.create(
    model="gpt-5",
    input=query,
    tools=[{
        "type": "web_search",
        "filters":{
            "allowed_domains": ["skyscanner.com", "expedia.com", "booking.com"]    
        }
    }]
)

print("Search Results: \n", response.output[-1].content[0].text)

In [ ]:
def flight_search(query: str) -> str:
    """
    Search for flights based on the provided query and optional domains.
    
    Args:
        query (str): The search query for flights.
        include_domains (list, optional): List of domains to include in the search.
        
    Returns:
        dict: Search results containing flight information.
    """
    response = openai_client.responses.create(
        model="gpt-5",
        input=query,
        tools=[{
            "type": "web_search",
            "filters":{
                "allowed_domains": ["skyscanner.com", "expedia.com", "booking.com"]    
            }
        }]
    )
    
    return response.output[-1].content[0].text

In [ ]:
query = "What are the best hotels in tokyo and osaka for tomorrow?"

# Hotel search tool
response = openai_client.responses.create(
    model="gpt-5",
    input=query,
    tools=[{
        "type": "web_search",
        "filters":{
            "allowed_domains": ["airbnb.com", "booking.com"]    
        }
    }]
)

print("Search Results: \n", response.output[-1].content[0].text)

In [ ]:
def hotel_search(query: str) -> dict:
    """
    Search for hotels based on the provided query and optional domains.
    
    Args:
        query (str): The search query for hotels.
        include_domains (list, optional): List of domains to include in the search.
        
    Returns:
        dict: Search results containing hotel information.
    """
    response = openai_client.responses.create(
        model="gpt-5",
        input=query,
        tools=[{
            "type": "web_search",
            "filters":{
                "allowed_domains": ["airbnb.com", "booking.com"]    
            }
        }]
    )
    return response.output[-1].content[0].text

## Step 3: Define our tools schema

In [ ]:
flight_tool_schema = function_to_tool(flight_search)
hotel_tool_schema = function_to_tool(hotel_search)

## Step 4: Prompt Template

In [ ]:
class PromptTemplate:
    def __init__(self, template: str, input_variables: list = [str]):
        self.template = template
        self.input_variables = input_variables

    def format(self, **kwargs) -> str:
        return self.template.format(**{k: kwargs[k] for k in self.input_variables})

In [ ]:
prompt = PromptTemplate(
    template="I want to travel to {destination} from {origin} on {departure_date} and return on {return_date}. What are the best options for {class_type} class",
    input_variables=["destination", "origin", "departure_date", "return_date", "class_type"]
)

In [ ]:
user_message = prompt.format(
    destination="Tokyo", origin="Abu Dhabi", departure_date="2025-10-05", return_date="2025-10-10", class_type="economy")

## Step 5: Call the OpenAI Responses API

In [ ]:
system_message = """

You are a helpful travel planner and your are to assist the user in planning their travel itinerary.

1. Use the provided details to search for the best flight and hotel options using the provided tools.
2. Summarize the results in a neatly formatted itinerary.
3. Do not ask the user any questions, just provide the itinerary based on the details provided.
4. Make the itinerary clear, helpful and easy to read.

Example flow:
- Receive the user's destination, travel dates, and preferences.
- Use your search tool to find the best flight options.
- Use your search tool to find the best hotel options.
- Summarize the results in a neatly formatted itinerary.

---
**Your Adventure Awaits!** ✈️🏨🌍
- Destination: {destination}
- Origin: {origin}
- Departure Date: {departure_date}
- Return Date: {return_date}
- Class Type: {class_type}

**Flight Options:**
{flight_options}

**Hotel Options:**
{hotel_options}

Have a fantastic trip!

Only return the final itinerary, do not include any other text.
"""

In [ ]:
input_list = [{"role": "system", "content": system_message},
              {"role": "user", "content": user_message}]

In [ ]:
def get_response(input_list):
    response = openai_client.responses.create(
        model="gpt-5",
        input=input_list,
        tools=[flight_tool_schema, hotel_tool_schema],
        tool_choice="auto",
        parallel_tool_calls=False
    )
    return response

In [ ]:
response = get_response(input_list)

In [ ]:
print(response.output[-1].arguments)
print(response.output[-1].name)
print(response.output[-1].type)
print(response.output[-1].call_id)

In [ ]:
#response = get_response(input_list)

if response.output[-1].type == "message":
    input_list.append({"role": "assistant", "content": response.output[-1]})
if response.output[-1].type == "function_call":
    input_list += response.output

In [ ]:
input_list

## Step 6: Handle tools calls

In [ ]:
def call_function(name, args):
    if name == "flight_search":
        return flight_search(**args)
    if name == "hotel_search":
        return hotel_search(**args)

In [ ]:
name = response.output[0].name
args = json.loads(response.output[0].arguments)

result = call_function(name, args)

input_list.append({
    "type": "function_call_output",
    "call_id": response.output[0].call_id,
    "output": str(result)
})


In [ ]:
input_list

## Step 7: The App Logic

In [ ]:
def trip_planner(destination, origin, departure_date, return_date, class_type):
    user_message = prompt.format(
        destination=destination, 
        origin=origin, 
        departure_date=departure_date, 
        return_date=return_date, 
        class_type=class_type)

    input_list = [{"role": "system", "content": system_message},
              {"role": "user", "content": user_message}]

    response = get_response(input_list)

    if response.output[0].type == "function_call":
        while response.output[0].type == "function_call":
            input_list += response.output
            name = response.output[0].name
            args = json.loads(response.output[0].arguments)
            
            result = call_function(name, args)

            input_list.append({
                "type": "function_call_output",
                "call_id": response.output[0].call_id,
                "output": str(result)
            })

            response = get_response(input_list)

    return response.output[-1].content[0].text

In [ ]:
trip_planner("Tokyo", "Abu Dhabi", "2025-10-05", "2025-10-10", "economy")

## Step 8: Gradio UI

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("# Travel Itinerary Planner ✈️🏨🌍")
    gr.Markdown("Plan your perfect trip with ease! Enter your travel details below and let our AI-powered planner create a personalized itinerary for you.")
    
    with gr.Row():
        destination = gr.Textbox(label="Destination", placeholder="Enter your travel destination (e.g., Tokyo)")
        origin = gr.Textbox(label="Origin", placeholder="Enter your departure city (e.g., Abu Dhabi)")

    with gr.Row():
        departure_date = gr.Textbox(label="Departure Date", placeholder="Enter your departure date (e.g., 2025-10-05)")
        return_date = gr.Textbox(label="Return Date", placeholder="Enter your return date (e.g., 2025-10-10)")

    with gr.Row():
        class_type = gr.Dropdown(choices=["economy", "business", "first"], label="Class Type", value="economy")
        
    plan_button = gr.Button("Plan My Trip")
        
    output = gr.Markdown(label="Your Itinerary will appear here...")
    
    def plan_trip(destination, origin, departure_date, return_date, class_type):
        itinerary = trip_planner(destination, origin, departure_date, return_date, class_type)
        return itinerary
    
    plan_button.click(fn=plan_trip, inputs=[destination, origin, departure_date, return_date, class_type], outputs=output)

demo.launch()